In [1]:
#import program modules
import finvest as fv

# **Final Project**
## Comp 4447 - Data Science Tools 1
 **By Brian McCullough**

**Main Process :**
I wanted to build a process that would take a company that a user is interested in which gives context on how that company measures up compared to their nearest competitors in the same space of the market.

**DS Tools 1 processes :**
From the course, I applied the web scraping, html reading, visualizations and some regex.

**Other MS of DS processes :**
From other portions of the program, I am logging steps and processes for when the developement would break, heatmap correlations 

## **Process Overview**
The user is asked for a ticker symbol of a publicly traded company.\
Some examples (but not limited to):\
\
MSFT - Microsoft - technology company\
META - Meta (owner of Facebook) - communications services\
NOC - Northrup Grumman - industrials\
USB - U.S. Bancorp - financials

The first process will then web-scrape data about the company into a dataframe.  This data is then used to identify the Sector and Industry that this company competes in as well as determines some other company financial health characteristics to keep the comparisons as 'like-to-like' as possible.  This focuses on the Market Cap and the Debt levels of the given company and then creates a range around those characteristics to create the list of competitors.\
\
`Assign a ticker to the given ticker variable`

In [2]:
user_stock = 'SFIX'

### Step 1: Pull basic stock information

In [3]:
main_stock_info = fv.stck_base_data(user_stock)

2023-08-05 20:37:26,576 - urllib3.connectionpool - DEBUG - Starting new HTTPS connection (1): finviz.com:443
2023-08-05 20:37:26,906 - urllib3.connectionpool - DEBUG - https://finviz.com:443 "GET /quote.ashx?t=SFIX HTTP/1.1" 200 None
2023-08-05 20:37:27,005 - root - INFO - get full company overview of given stock


In [4]:
main_stock_info[['Ticker\n\n','Company','Sector','Industry','P/E','Debt/Eq','ROA','ROE','ROI']]

,Ticker\n\n,Company,Sector,Industry,P/E,Debt/Eq,ROA,ROE,ROI
0,SFIX,"Stitch Fix, Inc.",Consumer Cyclical,Apparel Retail,-,0.00,-33.90%,-85.10%,-63.70%


### Step 2: Find similar companies to the given stock ticker

In [7]:
sector_stock_info = fv.screen_stcks(main_stock_info)

2023-08-05 20:38:00,779 - root - INFO - call screener data of similar stocks to main starter stock
2023-08-05 20:38:00,965 - urllib3.connectionpool - DEBUG - https://finviz.com:443 "GET /screener.ashx?v=151&f=sec_consumercyclical,ind_apparelretail,geo_usa&ft=4&c=0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70 HTTP/1.1" 200 None


2023-08-05 20:38:02,274 - urllib3.connectionpool - DEBUG - https://finviz.com:443 "GET /screener.ashx?v=151&f=sec_consumercyclical,ind_apparelretail,geo_usa&ft=4&r=21&c=0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70 HTTP/1.1" 200 None
2023-08-05 20:38:02,345 - root - INFO - extract Market Cap data to create lower limit for competitor comps


Some quick info on the companies that were filtered into our data

In [8]:
lngth_df = len(sector_stock_info)
print(f"# of companies that fit the filters = {lngth_df}")
print(f"Sector: {sector_stock_info['Sector'][0]}   ///   Industry: {sector_stock_info['Industry'][0]}")
print(f"Market cap range(min-max): {sector_stock_info['Market Cap'].min()//1e6}M - {sector_stock_info['Market Cap'].max()//1e9}B")
print(f"Debt/Eq ratio range: {sector_stock_info['Debt/Eq'].min()} - {sector_stock_info['Debt/Eq'].max()}")


# of companies that fit the filters = 20
Sector: Consumer Cyclical   ///   Industry: Apparel Retail
Market cap range(min-max): 62.0M - 3.0B
Debt/Eq ratio range: 0.0 - 0.37


In this example we have quite a few contenders for investment but we have no reference yet on how these companies are related in terms of financial quality and value in order to know where to spend our time researching into best opportunities for investing.\
So we need to *rank* our companies and see how they compare to each other.


In [9]:
ranked_stock_info = fv.rank_screener(sector_stock_info, user_stock)

In [17]:
print(ranked_stock_info.loc[ranked_stock_info['cat_grp'] == 1, ['Ticker\n\n','Company', 'cat_desc','Market Cap','P/E','P/S','P/B','P/FCF','EPS','ROA','ROE','ROI']])
print()
print(ranked_stock_info.loc[ranked_stock_info['cat_grp'] == 2, ['Ticker\n\n','Company','cat_desc','Market Cap','P/E','P/S','P/B','P/FCF','EPS','ROA','ROE','ROI']])
print()
print(ranked_stock_info.loc[ranked_stock_info['cat_grp'] == 3, ['Ticker\n\n','Company','cat_desc','Market Cap','P/E','P/S','P/B','P/FCF','EPS','ROA','ROE','ROI']])


   Ticker\n\n           Company cat_desc   Market Cap  P/E  P/S   P/B  P/FCF  \
18       SFIX  Stitch Fix, Inc.     Main  527890000.0  NaN  0.3  2.11    NaN   

     EPS    ROA    ROE    ROI  
18 -2.15 -0.339 -0.851 -0.637  

  Ticker\n\n                     Company  cat_desc   Market Cap   P/E   P/S  \
0       EXPR               Express, Inc.  Top Comp   62970000.0  0.27  0.04   
1       DXLG  Destination XL Group, Inc.  Top Comp  311700000.0  4.01  0.57   
2        GCO                Genesco Inc.  Top Comp  360730000.0  7.54  0.15   

    P/B  P/FCF   EPS    ROA    ROE    ROI  
0  0.27    NaN  3.19  0.181  1.821  0.748  
1  2.15   6.62  1.24  0.241  0.626  0.651  
2  0.59    NaN  3.88  0.032  0.083  0.116  

   Ticker\n\n           Company cat_desc    Market Cap    P/E   P/S  P/B  \
20      _AVG_  Stitch Fix, Inc.  Grp Avg  1.066280e+09  18.21  0.53  1.7   

    P/FCF   EPS   ROA   ROE   ROI  
20  33.02  2.13  0.04  0.18  0.16  
